<a href="https://colab.research.google.com/github/SangamSilwal/DeepLearning_DSeries/blob/main/D24_Tuning_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Bidirectional,Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [5]:
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
X_train = pad_sequences(X_train,maxlen=100)
X_test = pad_sequences(X_test,maxlen=100)
X_test.shape

(25000, 100)

In [15]:
def build_model(hp):
  model = Sequential()
  model.add(Embedding(10000,32,input_length=100))

  use_bidir = hp.Boolean("use_bidirectional")
  num_layers = hp.Int("num_layers",min_value=1,max_value=3,step=1)

  for i in range(num_layers):

    units = hp.Choice(f"units_{i}",[16,32,64,128])
    return_seq = True if i < num_layers -1 else False

    lstm_layer = LSTM(
        units=units,
        return_sequences=return_seq
    )

    if use_bidir:
      lstm_layer = Bidirectional(lstm_layer)

    model.add(lstm_layer)

  model.add(Dense(1,activation="sigmoid"))
  lr = hp.Choice("lr",[1e-4,5e-4,1e-3])
  model.compile(
      optimizer=Adam(lr),
      loss="binary_crossentropy",
      metrics=['accuracy']
  )
  return model

In [16]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    project_name="bidir_lstm_tuning",
)

tuner.search(X_train,y_train,epochs=10,validation_split=0.2)

Trial 10 Complete [00h 03m 01s]
val_accuracy: 0.8464000225067139

Best val_accuracy So Far: 0.8511999845504761
Total elapsed time: 00h 18m 56s


In [19]:
tuner.get_best_models(1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


<Sequential name=sequential, built=True>